In [ ]:
import pandas as pd
import numpy as np
import os

# Lecture 9 – Combining Data

## DSC 80, Spring 2022

### Announcements

- Discussion 3 is due (for extra credit!) **tomorrow at 11:59PM**.
- Lab 3 is due on **Monday, April 18th 11:59PM**.
    - Check [here](https://campuswire.com/c/G325FA25B/feed/507) for clarifications.
- Lab 1 (+more) grades are released – see [this post](https://campuswire.com/c/G325FA25B/feed/509) for details, and [this post](https://campuswire.com/c/G325FA25B/feed/508) for assignment solutions.
- Watch [this video 🎥](https://www.youtube.com/watch?v=uUawZfAgA64) for tips on how to work with the command-line.
- Project 2 will be released this weekend.

### Agenda

- Concatenating vertically.
- Concatenating horizontally.
- Joining and merging.
- Working with time series data.

Good resource: `pandas` [User Guide](https://pandas.pydata.org/docs/user_guide/merging.html).

## Recap: Concatenating vertically

### Example: Grades

By default, `pd.concat` stacks DataFrames row-wise, i.e. on top of one another.

In [ ]:
section_A = pd.DataFrame({
    'Name': ['Annie', 'Billy', 'Sally', 'Tommy'],
    'Midterm': [98, 82, 23, 45],
    'Final': [88, 100, 99, 67]
})

section_A

In [ ]:
section_B = pd.DataFrame({
    'Name': ['Junior', 'Rex', 'Flash'],
    'Midterm': [70, 99, 81],
    'Final': [42, 25, 90]
})

section_B

Let's use `pd.concat` on a list of the above two DataFrames.

In [ ]:
pd.concat([section_A, section_B])

### ⚠️ Warning: No loops!

- `pd.concat` returns a copy; it does not modify any of the input DataFrames.
- Do **not** use `pd.concat` in a loop, as it has terrible time and space efficiency.

```py
total = pd.DataFrame()
for df in dataframes:
    total = total.concat(df)
```

- Instead, use `pd.concat(dataframes)`, where `dataframes` is a list of DataFrames.

## Concatenating horizontally

### Example: Grades (again)

Suppose we have two DataFrames, `exams` and `assignments`, which both contain different attributes for the same individuals.

In [ ]:
exams = section_A.copy()
exams

In [ ]:
assignments = exams[['Name']].assign(Homeworks=[99, 45, 23, 81],
                                     Labs=[100, 100, 99, 100])

assignments

If we try to combine these DataFrames with `pd.concat`, we don't quite get what we're looking for.

In [ ]:
pd.concat([exams, assignments])

But that's where the `axis` argument becomes handy. 

Remember, most `pandas` operations default to `axis=0`, but here we want to concatenate the columns of `exams` to the columns of `assignments`, so we should use `axis=1`.

In [ ]:
pd.concat([exams, assignments], axis=1)

Note that the `'Name'` column appears twice!

### Concatenating horizontally

- To concatenate two DataFrames horizontally, use `pd.concat` with `axis=1`.
- Concatenation is done by matching indexes, regardless of their order. **It does not look at any column values!**

<center><img src='imgs/merging_concat_series_ignore_index.png' width='80%'></center>

Note that the call to `pd.concat` below works as expected, even though the orders of the names in `exams_by_name` and `assignments_by_name` are different.

In [ ]:
# .loc[::-1] reverses the rows of the DataFrame
exams_by_name = exams.set_index('Name').loc[::-1]
exams_by_name

In [ ]:
assignments_by_name = assignments.set_index('Name')
assignments_by_name

In [ ]:
pd.concat([exams_by_name, assignments_by_name], axis=1)

Remember that `pd.concat` only looks at the index when combining rows, not at any other columns.

In [ ]:
exams_reversed = exams.loc[::-1].reset_index(drop=True)
exams_reversed

In [ ]:
assignments

In [ ]:
pd.concat([exams_reversed, assignments], axis=1)

### Missing rows?

If we concatenate two DataFrames that don't share row indexes, `NaN`s are added in the rows that aren't shared.

In [ ]:
exams_extra = exams.copy()
exams_extra.loc[4] = ['Junior', 100, 100]
exams_extra

In [ ]:
assignments

In [ ]:
pd.concat([exams_extra, assignments], axis=1)

### Summary: `pd.concat`

- `pd.concat` "stitches" two or more DataFrames together.
- If you use `axis=0`, the DataFrames are concatenated **vertically** based on column names (rows on top of rows).
- If you use `axis=1`, the DataFrames are concatenated **horizontally** based on row indexes (columns next to columns).

## Joining and merging

### Concatenating horizontally

- `pd.concat` with `axis=1` combines DataFrames horizontally.
- To combine DataFrames horizontally in more advanced ways, we perform a **join** (also known as a **merge**).

### Joins

- A **join** creates a new DataFrame by combining the rows of two DataFrames.
- A join is appropriate when we have two sources of information
    - about the same individuals, that is
    - linked by a common column.
- The common column is called the **join key**.

### Example

Here, the join key is `'Player Id'`.

<center><img src="imgs/join.png" width="50%"></center>

### The `merge` method

- The `merge` DataFrame method joins two tables by columns or indexes.
    - "Merge" is just `pandas`' word for "join".
    - It also exists as a `pandas` function.
- If join keys are not specified, all shared columns between the two DataFrames are used by default.

Let's work with a small example.

In [ ]:
temps = pd.DataFrame({
    'City': ['San Diego', 'Toronto', 'Rome'],
    'Temperature': [76, 28, 56]
})

temps

In [ ]:
countries = pd.DataFrame({
    'City': ['Toronto', 'Shanghai', 'San Diego'],
    'Country': ['Canada', 'China', 'USA']
})

countries

In [ ]:
temps.merge(countries)

We didn't specify which columns to merge on, so it defaulted to `'City'`.

### Join types: inner joins

- Note that `'Rome'` and `'Shanghai'` do not appear in the merged DataFrame.
- This is because there is:
    - no city named `'Rome'` in the second DataFrame, and
    - no city named `'Shanghai'` in the first DataFrame.
- The default type of join that `merge` performs is an **inner join**, which keeps the **intersection** of the join keys.


<center><img src='imgs/image_0.png' width=40%></center>

### Different join types handle mismatches differently

There are four types of joins.

* **Inner:** keep **only** matching keys (intersection).
* **Outer:** keeps **all** keys in both DataFrames (union).
* **Left:** keep all keys in the left DataFrame, whether or not they are in the right DataFrame.
* **Right:** keep all keys in the right DataFrame, whether or not they are in the left DataFrame.

<center><img src='imgs/image_1.png' width=60%></center>

### Examples of join types

- To specify which type of join we want to perform, we use the `how` argument (the default is `how='inner'`).
- The left DataFrame is the DataFrame that **preceeds** `.merge`, and the right DataFrame is the argument to `merge`.
- Alternatively, you can use the `pd.merge` **function**, in which the first argument is the left DataFrame and the second argument is the right DataFrame.

In [ ]:
temps

In [ ]:
countries

Let's try an outer join.

In [ ]:
temps.merge(countries, how='outer')

In [ ]:
# merge is also a pandas function
pd.merge(temps, countries, how='outer')

Note the `NaN`s in the rows for `'Rome'` and `'Shanghai'`.

Also note that an outer join is what `pd.concat` does by default, when there are no duplicated keys in either DataFrame.

In [ ]:
pd.concat([temps.set_index('City'), countries.set_index('City')], axis=1)

Let's try left and right joins.

In [ ]:
temps

In [ ]:
countries

First, a left join.

In [ ]:
temps.merge(countries, how='left')

How about a right join?

In [ ]:
temps.merge(countries, how='right')

Note that `a.merge(b, how='left')` is the same as `b.merge(a, how='right')`. The only difference is the order of the columns in the result.

In [ ]:
countries.merge(temps, how='left')

### Specifying join keys

- `pandas` defaults to using the shared column(s) as join keys.
- If there are multiple shared column names and you only want to join on one of them, **or** if there are no shared column names, then you will need to specify which columns to join on.
- Two solutions:
    1.  Use the `on` argument if the desired column(s) have the same names in both DataFrames.
    2. Use the `left_on` or `left_index` argument AND the `right_on` or `right_index` argument.

In [ ]:
exams

In [ ]:
overall = pd.DataFrame({
    'PID': ['A15253545', 'A10348245', 'A13349069', 'A18485824', 'A10094857'],
    'Student': ['Billy', 'Sally', 'Annie', 'Larry', 'Johnny'],
    'Final': [88, 64, 91, 45, 89]
})

overall

This is not what we're looking for.

In [ ]:
exams.merge(overall)

Instead, we need to tell `pandas` to look in the `'Name'` column of `exams` and `'Student'` column of `overall`. 

In [ ]:
exams.merge(overall, left_on='Name', right_on='Student')

If there are shared column names in the two DataFrames you are merging **that you are not using as join keys**, by default `'_x'` and `'_y'` are appended to their names.

In [ ]:
exams.merge(overall, left_on='Name', right_on='Student', suffixes=('_Exam', '_Overall'))

If the desired join key is in the index, assign `left_index` or `right_index` to `True`.

In [ ]:
exams

In [ ]:
overall_by_student = overall.set_index('Student')
overall_by_student

In [ ]:
exams.merge(overall_by_student, left_on='Name', right_index=True, suffixes=('_Exam', '_Overall'))

## Many-to-one & many-to-many joins

### One-to-one joins

- So far in this lecture, the joins we have worked with are called **one-to-one** joins.
- Neither the left DataFrame nor the right DataFrame contained any duplicates in the join key.
- What if there are duplicated join keys, in one or both of the DataFrames we are merging?

### Many-to-one joins

- Many-to-one joins are joins where **one** of the DataFrames contains duplicate values in the join key. 
- The resulting DataFrame will preserve those duplicate entries as appropriate. 

In [ ]:
profs = pd.DataFrame(
[['Brad', 'UCB', 8],
 ['Janine', 'UCSD', 7],
 ['Marina', 'UIC', 6],
 ['Justin', 'OSU', 4],
 ['Aaron', 'UCB', 4],
 ['Soohyun', 'UCSD', 1],
 ['Suraj', 'UCB', 1]],
    columns=['Name', 'School', 'Years']
)

profs

In [ ]:
schools = pd.DataFrame({
    'Abr': ['UCSD', 'UCLA', 'UCB', 'UIC'],
    'Full': ['University of California, San Diego', 'University of California, Los Angeles', 'University of California, Berkeley', 'University of Illinois Chicago']
})

schools

Note that when merging `profs` and `schools`, the information from `schools` is duplicated (`'University of California, San Diego'` appears twice and `'University of California, Berkeley'` appears three times).

In [ ]:
profs.merge(schools, left_on='School', right_on='Abr', how='left')

### Many-to-many joins

Many-to-many joins are joins where both DataFrames have duplicate values in the join key.

In [ ]:
profs

In [ ]:
programs = pd.DataFrame({
    'uni': ['UCSD', 'UCSD', 'UCSD', 'UCB', 'OSU', 'OSU'],
    'dept': ['Math', 'HDSI', 'COGS', 'CS', 'Math', 'CS'],
    'grad_students': [205, 54, 281, 439, 304, 193]
})

programs

Before running the following cell, try predicting the number of rows in the output.

In [ ]:
profs.merge(programs, left_on='School', right_on='uni')

- `merge` stitched together every UCSD row in `profs` with every UCSD row in `programs`. 
- Since there were 2 UCSD rows in `profs` and 3 in `programs`, there are $2 \cdot 3 = 6$ UCSD rows in the output. The same applies for all other schools.

## Example: SDPD vehicle stops

### Aside: accessing file names programmatically

- At times, you'll need to load in all of the files in a given folder.
- `os.listdir(dirname)` returns a **list** of the names of the files in the folder `dirname`.

In [ ]:
os.listdir('data')

- Sometimes, you'll want to extract file names that follow a specific pattern. The `pathlib` library allows you to do this.

In [ ]:
import pathlib
file_list = list(pathlib.Path().glob('data/stops*.csv')) # glob allows for pattern matching
file_list

- You can accomplish something similar in the command-line.
- Place a `!` in front of a command in a Jupyter Notebook cell to run it on the command-line.

In [ ]:
!ls data/stops*.csv

### Loading in the data

In [ ]:
file_list

In [ ]:
list_of_dfs = [pd.read_csv(file) for file in file_list]

In [ ]:
for df in list_of_dfs:
    display(df.head())

We need to concatenate these two DataFrames **vertically**.

In [ ]:
stops = pd.concat(list_of_dfs, ignore_index=True)

In [ ]:
stops.head()

In [ ]:
len(stops)

### Investigating races

Right now, `'subject_race'` is stored as a single character. What does `'I'` mean? `'H'`?

In [ ]:
stops.head()

In [ ]:
stops['subject_race'].value_counts(normalize=True)

Fortunately, we have access to another dataset that describes each of the race codes.

In [ ]:
races = pd.read_csv('data/race_codes.csv')
races

Let's join the distribution of races with the DataFrame of race codes.

**Question:** Is this a one-to-one join?

In [ ]:
race_percentages = stops['subject_race'].value_counts(normalize=True).rename('Proportion').to_frame()
race_percentages.merge(races, left_index=True, right_on='Race Code')

The **level of granularity** of the races in our data right now seems inconsistent. For instance, `'WHITE'` and `'BLACK'` are much more broad than `'FILIPINO'`, `'JAPANESE'`, and `'GUAMANIAN'`.

### Adjusting granularity

Let's try and adjust our race data so that we have a consistent level of granularity. Here's what we want to create:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Race Code</th>
      <th>Description</th>
      <th>Race_Category</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>A</td>
      <td>OTHER ASIAN</td>
      <td>Asian</td>
    </tr>
    <tr>
      <th>1</th>
      <td>B</td>
      <td>BLACK</td>
      <td>Black</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C</td>
      <td>CHINESE</td>
      <td>Asian</td>
    </tr>
    <tr>
      <th>3</th>
      <td>D</td>
      <td>CAMBODIAN</td>
      <td>Asian</td>
    </tr>
    <tr>
      <th>4</th>
      <td>F</td>
      <td>FILIPINO</td>
      <td>Asian</td>
    </tr>
  </tbody>
</table>

We can do this by manually defining a mapping between race codes and desired categories.

In [ ]:
race_dict = {'A':'Asian',
             'B':'Black',
             'C':'Asian',
             'D':'Asian',
             'F':'Asian',
             'G':'Asian',
             'H':'Hispanic',
             'I':'Native American',
             'J':'Asian',
             'K':'Asian',
             'L':'Asian',
             'O':'Other',
             'P':'Asian',
             'S':'Asian',
             'U':'Hawaiian',
             'V':'Asian',
             'W':'White',
             'Z':'Asian'
            }

There are two ways to replace all of the `'Race Code'`s in `races` with the above categories:
- Use the Series `replace` method.
- Convert the above mapping to a DataFrame and join it with `races`.

Joining requires sorting, where as replacing does not. Let's go with the first option.

In [ ]:
races['Race_Category'] = races['Race Code'].replace(race_dict)
races

Now, we need to join `stops` with `races`. An important question is, what type of join should we use (inner, outer, left, right)?

In [ ]:
stops.head()

In [ ]:
stops['subject_race'].isna().mean()

So that we don't discard the individuals whose races we don't have, we will use a **left join**.

In [ ]:
stops_merged = stops.merge(races, left_on='subject_race', right_on='Race Code', how='left')
stops_merged.head()

Now we can compute a more meaningful distribution of races.

In [ ]:
dist = stops_merged['Race_Category'].value_counts(normalize=True)
dist

In [ ]:
dist.plot(kind='bar', figsize=(10, 5));

That's a bit more helpful.

## Aside: Working with time series data

### Time series – why now?

- Data is often partitioned by time. For instance, there may be one `.csv` file per day for 1 year.
- To combine the datasets, we will need to load in the files as DataFrames and `pd.concat` the DataFrames together.
- Note: "time series" is a general term and is not related to Series in `pandas`.

### Datetime types

When working with time data, you will see two different kinds of "times":

* **Datetimes** reference particular moments in time (e.g. November 26th, 1998 at 8:26AM).
    - Could just be a date, e.g. September 15, 2014.
    - Could just be a time, e.g. 4:45 AM.
    - Datetimes typically don't keep track of timezones.
* **Timedeltas**, or durations, reference an exact length of time (e.g. a duration of 3 hours).

### The `datetime` module

Python has an in-built `datetime` module, which contains `datetime` and `timedelta` types. These are much more convenient to deal with than strings that contain times.

In [ ]:
import datetime

In [ ]:
datetime.datetime.now()

In [ ]:
datetime.datetime.now() + datetime.timedelta(days=3, hours=5)

Recall, Unix timestamps count the number of seconds since January 1st, 1970.

In [ ]:
datetime.datetime.now().timestamp()

### Times in `pandas`

- `pd.Timestamp` is the `pandas` equivalent of `datetime`.
- `pd.to_datetime` converts strings to `pd.Timestamp` objects.

In [ ]:
pd.Timestamp(year=1998, month=11, day=26)

In [ ]:
final_start = pd.to_datetime('June 4th, 2022, 11:30AM')
final_start

In [ ]:
final_finish = pd.to_datetime('June 4th, 2022, 2:30PM')
final_finish

Timestamps have time-related attributes, e.g. `dayofweek`, `hour`, `min`, `sec`.

In [ ]:
final_finish.dayofweek

In [ ]:
final_finish.year

Subtracting timestamps yields `pd.Timedelta` objects.

In [ ]:
final_finish - final_start

### Timestamps in DataFrames

- If we create a Series of datetimes with `pd.to_datetime`, `pandas` stores them as yet *another* type:
`np.datetime64`.
    - These are similar to `pd.Timestamp`, but optimized for memory and speed efficiency.
- If we access a single time, we get a `pd.Timestamp` back.
- See [the documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html) for more details.

In [ ]:
times = pd.DataFrame({'finish': pd.to_datetime(['Sun, Jan 01, 1989', 
                                                '2022-04-15T11:00', 
                                                '1/1/1970'])})
times

In [ ]:
times.info()

In [ ]:
times.iloc[0, 0]

In [ ]:
times.sort_values('finish')

### Example: Exam speeds

Below, we have the Final Exam starting and ending times for two sections of a course.

In [ ]:
times_A = pd.DataFrame({
    'Name': ['Annie', 'Billy', 'Sally', 'Tommy'],
    'start_exam': ['15:00', '15:02', '15:01', '15:00'],
    'finish_exam': ['16:00', '17:58', '17:05', '16:55']
})

times_B = pd.DataFrame({
    'Name': ['Junior', 'Rex', 'Flash'],
    'start_exam': ['18:00', '18:06', '19:07'],
    'finish_exam': ['20:00', '20:50', '20:59']
})

display(times_A)
display(times_B)

**Question:** Who finished the exam the fastest amongst all students in the course?

Approach:
1. Concatenate the two DataFrames.
2. Convert the time columns to `pd.Timestamp`.
3. Find the difference between `'finish_exam'` and `'start_exam'`.
4. Sort.
5. Pick the fastest exam taker.

In [ ]:
# Step 1
both_versions = pd.concat([times_A, times_B])
both_versions

In [ ]:
# Step 2
both_versions = both_versions.assign(
    start_exam=pd.to_datetime(both_versions['start_exam']),
    finish_exam=pd.to_datetime(both_versions['finish_exam'])
)

both_versions.info()

In [ ]:
# Step 3
both_versions = both_versions.assign(
    elapsed=both_versions['finish_exam'] - both_versions['start_exam']
)

both_versions

In [ ]:
# Steps 4 and 5
both_versions.sort_values('elapsed').iloc[0].loc['Name']

## Summary, next time

### Summary

- `pd.concat` "stitches" two or more DataFrames together, either vertically or horizontally.
    - Vertically: looks at column names. Horizontally: looks at row indexes.
- The `merge` DataFrame method **joins** two DataFrames together based on a shared column, called a join key. There are four types of joins:
    - Inner join: keeps the **intersection** of the join keys.
    - Outer join: keeps the **union** of the join keys.
    - Left/right joins: keeps all of the join keys in the left/right DataFrame.
    - In outer/left/right joins, all missing fields are filled with `NaN`s.
- Timestamps in `pandas` are stored using `pd.Timestamp` and `pd.Timedelta` objects.
- **Next time:** Permutation testing!